In [ ]:
# =======================================================
# TPS October 2021 - Baseline
# =======================================================
# Name: Bárbara Sulpis
# Date: 01-oct-2021
# Description: I want to run a basic model to see what is the base ROC value without doing any angineering
# Python 3 - kaggle/python Docker image: https://github.com/kaggle/docker-python

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from sklearn.model_selection import train_test_split

#Lgbm
import lightgbm as lgb

# roc
import sklearn.metrics as metrics   # Para la curva ROC
import matplotlib.pyplot as plt     # Para la curva ROC


# ---------------------------
# Input data:
# Go to file -> add or upload data -> "Competition" data tab and select the commpetition which you want to add the csv data data "
# files are available in the read-only "../input/" directory
# ---------------------------
#list =  os. getcwd()
#print(list)
#os. chdir("kaggle") 

list =  os. getcwd()
print(list) # shoud be in "kaggle" directory

# I left this commented if you want to check that the files are there
# i = 0
# for subdir, dirs, files in os.walk('./'):
#     for file in files:
#         print(file)
#         i+= 1
#         if i>20: 
#             break


data = pd.read_csv("../input/tabular-playground-series-oct-2021/train.csv")        
subm = pd.read_csv("../input/tabular-playground-series-oct-2021/test.csv")        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def Draw_ROC(y_test, preds, base_name):
    fpr, tpr, threshold = metrics.roc_curve(y_test , preds)
    roc_auc = metrics.auc(fpr, tpr)

    plt.title('Receiver Operating Characteristic ('+ base_name+ ')')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.6f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()


In [ ]:
data

In [ ]:
# --------------------------------------------------------------------
#     SPLIT IN TARGET AND DATA
# --------------------------------------------------------------------
y = data['target']
X = data.drop(['id', 'target'], axis=1)

In [ ]:
# --------------------------------------------------------------------
#     SPLIT TRAIN - TEST - VALIDATION  70%-15%-15%
# --------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=123)

In [ ]:
# -----------------------------------------
# LGBM Classification
# -----------------------------------------

# Use a parametrization that worked fine in some projects
params = {'colsample_bytree': 0.541999392404749, 
          'min_child_samples': 430, 
          'min_child_weight': 10.0, 
          'num_leaves': 44,
          'reg_alpha': 1, 
          'reg_lambda': 100, 
          'subsample': 0.7653033041096617}

#n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
clf = lgb.LGBMClassifier(max_depth=-1, silent=True, metric='None', n_jobs=-1, n_estimators=5000, random_state=314)

# This allows us to avoid overtraining and we do not need to optimise the number of trees
fit_params={"early_stopping_rounds":30, 
            "eval_metric" : 'auc', 
            "eval_set" : [(X_test, y_test)],
            'eval_names': ['valid'],
            #'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_099)],
            'verbose': 100,
            'categorical_feature': 'auto'}

# opt_parameters = {
#     'colsample_bytree': params['colsample_bytree'], 
#     'min_child_samples': params['min_child_samples'], 
#     'min_child_weight': params['min_child_weight'], 
#     'num_leaves': params['num_leaves'], 
#     'reg_alpha': params['reg_alpha'], 
#     'reg_lambda': params['reg_lambda'], 
#     'subsample': params['subsample']
# }

#Configure from the HP optimisation
#clf_final = lgb.LGBMClassifier(**gs.best_estimator_.get_params())

#Configure locally from hardcoded values
clf_final = lgb.LGBMClassifier(**clf.get_params())

#set optimal parameters
clf_final.set_params(**params)

#Train the final model with learning rate decay
clf_final.fit(X_train, y_train, **fit_params )  #, callbacks=[lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_0995)])

In [ ]:
# ---------------------------------------
#  ROC : TRAIN
# ---------------------------------------

# calculate the fpr and tpr for all thresholds of the classification
y_predict_xgr = clf_final.predict_proba(X_train)
preds = y_predict_xgr  [:,1]   # CLF_pred es la variable con las predicciones de probabilidad
base_name = 'TRAIN'
Draw_ROC(y_train, preds, base_name)

In [ ]:
# ---------------------------------------
#  ROC : TEST
# ---------------------------------------

# calculate the fpr and tpr for all thresholds of the classification
y_predict_xgr = clf_final.predict_proba(X_test)
preds = y_predict_xgr  [:,1]   # CLF_pred es la variable con las predicciones de probabilidad
base_name = 'TEST'
Draw_ROC(y_test, preds, base_name)

In [ ]:
# ---------------------------------------
#  ROC : VALIDATION
# ---------------------------------------

# calculate the fpr and tpr for all thresholds of the classification
y_predict_xgr = clf_final.predict_proba(X_val)
preds = y_predict_xgr  [:,1]   # CLF_pred es la variable con las predicciones de probabilidad
base_name = 'VALIDATION'
Draw_ROC(y_val, preds, base_name)

In [ ]:
# ---------------------------------------
#  SUBMISSION FILE 
# ---------------------------------------
X_subm = subm.drop(['id'], axis=1)
y_preds_subm = clf_final.predict_proba(X_subm)

submit = pd.read_csv("../input/tabular-playground-series-oct-2021/sample_submission.csv")
submit['target'] = y_preds_subm  [:,1]
#submit.to_csv("LGBM_Baseline.csv", index=False)
submit.head()

In [ ]:
submit.to_csv("LGBM_Baseline.csv", index=False)
